# Task 3 : Échantillonnage et Hypothèses FD

## Objectif
Étudier comment les dépendances fonctionnelles suggérées par un LLM sur des **échantillons** peuvent différer de celles qui tiennent sur le **dataset complet**.

## Ce qu'on doit faire :
1. Créer des échantillons (max 50 lignes) : aléatoire et stratifié
2. Montrer les échantillons au LLM et lui demander de suggérer des FDs
3. Vérifier si ces FDs tiennent sur l'échantillon ET sur le dataset complet
4. Identifier les faux positifs et les FDs trompeuses

## Idée clé
> L'échantillonnage crée des **hypothèses**, pas des **vérités**.

In [ ]:
# Configuration de Claude (Anthropic)
import anthropic
import json
import pandas as pd
import numpy as np
from IPython.display import display, Markdown
import time
import os
from dotenv import load_dotenv

# Charger les variables d'environnement depuis le fichier .env
# Force le rechargement
load_dotenv(override=True)

# Récupérer la clé API Claude depuis les variables d'environnement
CLAUDE_API_KEY = os.getenv("CLAUDE_API_KEY")

# Debug : afficher les premiers et derniers caractères de la clé
if CLAUDE_API_KEY:
    print(f"🔑 Clé chargée : {CLAUDE_API_KEY[:15]}...{CLAUDE_API_KEY[-5:]}")
    print(f"📏 Longueur de la clé : {len(CLAUDE_API_KEY)} caractères")
else:
    raise ValueError(
        "❌ Clé API Claude introuvable.\n"
        "➡️ Vérifie que :\n"
        "1. Le fichier .env existe\n"
        "2. Il contient la ligne : CLAUDE_API_KEY=ta_cle_api\n"
        "3. Le fichier .env est bien chargé"
    )

# Configurer le client Claude
client = anthropic.Anthropic(api_key=CLAUDE_API_KEY)

# Test rapide de la connexion
try:
    test_response = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=10,
        messages=[{"role": "user", "content": "Dis juste 'OK'"}]
    )
    print(f"✅ Claude configuré avec succès ! Test: {test_response.content[0].text}")
except Exception as e:
    print(f"❌ Erreur de connexion : {e}")

In [ ]:
# Charger les datasets
datasets = {}

# IRIS
iris_path = '../Datasets/iris/iris.data'
if os.path.exists(iris_path):
    datasets['iris'] = pd.read_csv(iris_path, header=None, 
                                    names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class'])
    print(f"✅ iris: {datasets['iris'].shape}")
else:
    print(f"❌ iris non trouvé: {iris_path}")

# BRIDGES
bridges_path = '../Datasets/pittsburgh+bridges/bridges.data.version1'
if os.path.exists(bridges_path):
    datasets['bridges'] = pd.read_csv(bridges_path, header=None,
                                       names=['IDENTIF', 'RIVER', 'LOCATION', 'ERECTED', 'PURPOSE', 
                                              'LENGTH', 'LANES', 'CLEAR-G', 'T-OR-D', 'MATERIAL',
                                              'SPAN', 'REL-L', 'TYPE'])
    print(f"✅ bridges: {datasets['bridges'].shape}")
else:
    print(f"❌ bridges non trouvé: {bridges_path}")

# ABALONE
abalone_path = '../Datasets/abalone/abalone.data'
if os.path.exists(abalone_path):
    datasets['abalone'] = pd.read_csv(abalone_path, header=None,
                                       names=['Sex', 'Length', 'Diameter', 'Height', 'Whole_weight',
                                              'Shucked_weight', 'Viscera_weight', 'Shell_weight', 'Rings'])
    print(f"✅ abalone: {datasets['abalone'].shape}")
else:
    print(f"❌ abalone non trouvé: {abalone_path}")

# BREAST CANCER
bc_path = '../Datasets/breast+cancer+wisconsin+original/breast-cancer-wisconsin.data'
if os.path.exists(bc_path):
    datasets['breast-cancer'] = pd.read_csv(bc_path, header=None,
                                             names=['id', 'clump_thickness', 'uniformity_cell_size',
                                                    'uniformity_cell_shape', 'marginal_adhesion',
                                                    'single_epithelial_cell_size', 'bare_nuclei',
                                                    'bland_chromatin', 'normal_nucleoli', 'mitoses', 'class'])
    print(f"✅ breast-cancer: {datasets['breast-cancer'].shape}")
else:
    print(f"❌ breast-cancer non trouvé: {bc_path}")

print(f"\n📊 Datasets chargés: {list(datasets.keys())}")

## 2. Fonctions d'échantillonnage

Nous allons créer 3 types d'échantillons :
- **Aléatoire** : sélection au hasard
- **Stratifié** : garde les proportions de chaque groupe
- **Biaisé** : sur-représente une catégorie (pour montrer l'impact du biais)

In [ ]:
def random_sample(df, n=50, seed=42):
    """
    Échantillon aléatoire simple.
    """
    n = min(n, len(df))
    return df.sample(n=n, random_state=seed)

def stratified_sample(df, stratify_col, n=50, seed=42):
    """
    Échantillon stratifié basé sur une colonne.
    Garde les proportions de chaque groupe.
    """
    n = min(n, len(df))
    # Calculer les proportions
    proportions = df[stratify_col].value_counts(normalize=True)
    
    samples = []
    for value, prop in proportions.items():
        group = df[df[stratify_col] == value]
        group_n = max(1, int(n * prop))  # Au moins 1 par groupe
        group_n = min(group_n, len(group))
        samples.append(group.sample(n=group_n, random_state=seed))
    
    result = pd.concat(samples)
    # Ajuster si on a trop ou pas assez
    if len(result) > n:
        result = result.sample(n=n, random_state=seed)
    return result

def biased_sample(df, bias_col, bias_value, n=50, seed=42):
    """
    Échantillon biaisé : sur-représente une valeur particulière.
    """
    n = min(n, len(df))
    biased = df[df[bias_col] == bias_value]
    others = df[df[bias_col] != bias_value]
    
    # 70% du groupe biaisé, 30% des autres
    n_biased = min(int(n * 0.7), len(biased))
    n_others = min(n - n_biased, len(others))
    
    sample_biased = biased.sample(n=n_biased, random_state=seed) if n_biased > 0 else pd.DataFrame()
    sample_others = others.sample(n=n_others, random_state=seed) if n_others > 0 else pd.DataFrame()
    
    return pd.concat([sample_biased, sample_others])

print("✅ Fonctions d'échantillonnage définies")

## 3. Créer les échantillons

In [ ]:
# Créer les échantillons pour chaque dataset
samples = {}

# IRIS - échantillon aléatoire et stratifié par classe
if 'iris' in datasets:
    df = datasets['iris']
    samples['iris'] = {
        'random': random_sample(df, n=30),
        'stratified': stratified_sample(df, 'class', n=30)
    }
    print(f"✅ Échantillons iris créés (colonne stratifiée: class)")

# BRIDGES - échantillon aléatoire et stratifié par MATERIAL
if 'bridges' in datasets:
    df = datasets['bridges']
    samples['bridges'] = {
        'random': random_sample(df, n=40),
        'stratified': stratified_sample(df, 'MATERIAL', n=40)
    }
    print(f"✅ Échantillons bridges créés (colonne stratifiée: MATERIAL)")

# ABALONE - échantillon aléatoire et stratifié par Sex
if 'abalone' in datasets:
    df = datasets['abalone']
    samples['abalone'] = {
        'random': random_sample(df, n=50),
        'stratified': stratified_sample(df, 'Sex', n=50)
    }
    print(f"✅ Échantillons abalone créés (colonne stratifiée: Sex)")

# BREAST CANCER - échantillon aléatoire et stratifié par class
if 'breast-cancer' in datasets:
    df = datasets['breast-cancer']
    samples['breast-cancer'] = {
        'random': random_sample(df, n=50),
        'stratified': stratified_sample(df, 'class', n=50)
    }
    print(f"✅ Échantillons breast-cancer créés (colonne stratifiée: class)")

print(f"\n📊 Échantillons créés pour: {list(samples.keys())}")

In [ ]:
# Afficher un aperçu des échantillons
for dataset_name, dataset_samples in samples.items():
    print(f"\n{'='*60}")
    print(f"Dataset: {dataset_name.upper()}")
    print(f"{'='*60}")
    for sample_type, sample_df in dataset_samples.items():
        print(f"\n--- {sample_type.upper()} ({len(sample_df)} lignes) ---")
        display(sample_df.head(5))

## 4. Fonction pour interroger le LLM sur les FDs

In [ ]:
def ask_llm_for_fds(sample_df, dataset_name, sample_type):
    """
    Montre un échantillon au LLM (Claude) et lui demande de suggérer des FDs.
    """
    # Convertir l'échantillon en texte (max 20 lignes pour le prompt)
    sample_text = sample_df.head(20).to_string(index=False)
    columns = list(sample_df.columns)
    
    prompt = f"""Tu es un expert en bases de données. Voici un échantillon de données du dataset "{dataset_name}".

Colonnes: {columns}

Échantillon ({len(sample_df)} lignes, voici les 20 premières):
{sample_text}

En analysant CET ÉCHANTILLON UNIQUEMENT, quelles dépendances fonctionnelles (FDs) semblent tenir ?

Une FD X → Y signifie : si deux lignes ont la même valeur pour X, elles ont la même valeur pour Y.

Liste exactement 5 FDs que tu penses vraies dans cet échantillon.
Format de réponse (une FD par ligne):
FD1: colonne1 -> colonne2
FD2: colonne1, colonne2 -> colonne3
etc.

Ne donne QUE les FDs, pas d'explications."""
    
    try:
        message = client.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=1024,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return message.content[0].text
    except Exception as e:
        return f"ERREUR: {str(e)}"

print("✅ Fonction ask_llm_for_fds() définie (utilise Claude 3 Haiku)")

## 5. Interroger le LLM pour chaque échantillon

In [ ]:
import time

# Collecter les FDs suggérées par le LLM
llm_suggested_fds = {}

print("="*80)
print("INTERROGATION DU LLM POUR SUGGÉRER DES FDs")
print("="*80)

for dataset_name, dataset_samples in samples.items():
    llm_suggested_fds[dataset_name] = {}
    
    for sample_type, sample_df in dataset_samples.items():
        print(f"\n🔄 {dataset_name} - {sample_type}...")
        
        response = ask_llm_for_fds(sample_df, dataset_name, sample_type)
        llm_suggested_fds[dataset_name][sample_type] = response
        
        print(f"📝 Réponse du LLM:")
        print(response)
        print("-" * 40)
        
        time.sleep(2)  # Pause pour l'API

print("\n✅ Toutes les suggestions collectées !")

## 6. Parser les FDs suggérées

In [ ]:
import re

def parse_fd_response(response_text):
    """
    Extrait les FDs de la réponse du LLM.
    Format attendu: "colA, colB -> colC" ou "colA -> colB"
    """
    fds = []
    lines = response_text.strip().split('\n')
    
    for line in lines:
        # Chercher le pattern "X -> Y" ou "X → Y"
        match = re.search(r'([^->→]+)\s*[-→>]+\s*([^->→]+)', line)
        if match:
            lhs = match.group(1).strip()
            rhs = match.group(2).strip()
            
            # Nettoyer (enlever "FD1:", etc.)
            lhs = re.sub(r'^FD\d*:\s*', '', lhs)
            
            # Parser le LHS (peut être "col1, col2")
            lhs_cols = [c.strip() for c in lhs.split(',')]
            rhs_col = rhs.strip()
            
            fds.append({
                'lhs': lhs_cols,
                'rhs': rhs_col,
                'fd_string': f"{', '.join(lhs_cols)} -> {rhs_col}"
            })
    
    return fds

# Parser toutes les réponses
parsed_fds = {}
for dataset_name, dataset_responses in llm_suggested_fds.items():
    parsed_fds[dataset_name] = {}
    for sample_type, response in dataset_responses.items():
        parsed_fds[dataset_name][sample_type] = parse_fd_response(response)

# Afficher les FDs parsées
print("FDs suggérées par le LLM:")
for dataset_name, dataset_fds in parsed_fds.items():
    print(f"\n{dataset_name.upper()}:")
    for sample_type, fds in dataset_fds.items():
        print(f"  {sample_type}: {len(fds)} FDs")
        for fd in fds:
            print(f"    - {fd['fd_string']}")

## 7. Fonction de vérification des FDs

In [ ]:
def check_fd_holds(df, lhs_cols, rhs_col):
    """
    Vérifie si une FD tient dans un DataFrame.
    Retourne: (holds: bool, violations: int, total_groups: int)
    """
    # Vérifier que les colonnes existent
    for col in lhs_cols + [rhs_col]:
        if col not in df.columns:
            return None, 0, 0  # Colonne non trouvée
    
    # Grouper par LHS et compter les valeurs uniques de RHS
    grouped = df.groupby(lhs_cols)[rhs_col].nunique()
    
    # Une FD tient si chaque groupe a exactement 1 valeur pour RHS
    violations = (grouped > 1).sum()
    total_groups = len(grouped)
    holds = violations == 0
    
    return holds, violations, total_groups

def check_fd_approximate(df, lhs_cols, rhs_col):
    """
    Calcule le taux de validité approximatif d'une FD.
    Retourne le pourcentage de groupes qui respectent la FD.
    """
    for col in lhs_cols + [rhs_col]:
        if col not in df.columns:
            return None
    
    grouped = df.groupby(lhs_cols)[rhs_col].nunique()
    valid_groups = (grouped == 1).sum()
    total_groups = len(grouped)
    
    if total_groups == 0:
        return 0
    
    return (valid_groups / total_groups) * 100

print("✅ Fonctions de vérification définies")

## 8. Valider les FDs sur échantillons vs dataset complet

In [ ]:
# Valider chaque FD suggérée
validation_results = []

print("="*100)
print("VALIDATION DES FDs SUGGÉRÉES")
print("="*100)

for dataset_name, dataset_fds in parsed_fds.items():
    if dataset_name not in datasets:
        continue
    
    full_df = datasets[dataset_name]
    
    print(f"\n{'='*60}")
    print(f"Dataset: {dataset_name.upper()} ({len(full_df)} lignes)")
    print(f"{'='*60}")
    
    for sample_type, fds in dataset_fds.items():
        sample_df = samples[dataset_name][sample_type]
        
        print(f"\n--- FDs du {sample_type} ({len(sample_df)} lignes) ---")
        
        for fd in fds:
            lhs = fd['lhs']
            rhs = fd['rhs']
            fd_str = fd['fd_string']
            
            # Vérifier sur l'échantillon
            holds_sample, viol_sample, groups_sample = check_fd_holds(sample_df, lhs, rhs)
            
            # Vérifier sur le dataset complet
            holds_full, viol_full, groups_full = check_fd_holds(full_df, lhs, rhs)
            
            # Calculer la validité approximative
            approx_sample = check_fd_approximate(sample_df, lhs, rhs)
            approx_full = check_fd_approximate(full_df, lhs, rhs)
            
            if holds_sample is None or holds_full is None:
                print(f"  ⚠️ {fd_str} - Colonnes non trouvées")
                continue
            
            # Déterminer le statut
            if holds_sample and holds_full:
                status = "✅ VRAIE (échantillon ET complet)"
                category = "true_positive"
            elif holds_sample and not holds_full:
                status = "❌ FAUX POSITIF (vraie sur échantillon, fausse sur complet)"
                category = "false_positive"
            elif not holds_sample and holds_full:
                status = "🔶 Fausse sur échantillon mais vraie sur complet"
                category = "sample_error"
            else:
                status = "❌ FAUSSE (échantillon ET complet)"
                category = "false_negative"
            
            print(f"\n  📌 {fd_str}")
            print(f"     Échantillon: {holds_sample} ({approx_sample:.1f}% valide, {viol_sample} violations)")
            print(f"     Complet: {holds_full} ({approx_full:.1f}% valide, {viol_full} violations)")
            print(f"     {status}")
            
            validation_results.append({
                'dataset': dataset_name,
                'sample_type': sample_type,
                'fd': fd_str,
                'holds_sample': holds_sample,
                'holds_full': holds_full,
                'approx_sample': approx_sample,
                'approx_full': approx_full,
                'violations_full': viol_full,
                'category': category
            })

## 9. Tableau récapitulatif

In [ ]:
# Créer le tableau récapitulatif
if validation_results:
    results_df = pd.DataFrame(validation_results)
    
    print("\n" + "="*100)
    print("TABLEAU RÉCAPITULATIF")
    print("="*100)
    
    display(results_df[['dataset', 'sample_type', 'fd', 'holds_sample', 'holds_full', 'category']])
    
    # Statistiques
    print("\n📊 STATISTIQUES:")
    print(f"   Total FDs analysées: {len(results_df)}")
    print(f"   ✅ Vraies positives: {(results_df['category'] == 'true_positive').sum()}")
    print(f"   ❌ Faux positifs: {(results_df['category'] == 'false_positive').sum()}")
    print(f"   ❌ Fausses: {(results_df['category'] == 'false_negative').sum()}")
else:
    print("Aucun résultat à afficher")

## 10. Analyse des Faux Positifs

In [ ]:
# Analyser les faux positifs
print("\n" + "="*80)
print("ANALYSE DES FAUX POSITIFS")
print("="*80)
print("\nCes FDs semblaient vraies sur l'échantillon mais sont fausses sur le dataset complet.")
print("Cela montre que l'échantillonnage cache des violations !\n")

if validation_results:
    false_positives = [r for r in validation_results if r['category'] == 'false_positive']
    
    if false_positives:
        for i, fp in enumerate(false_positives, 1):
            print(f"\n--- Faux Positif {i} ---")
            print(f"Dataset: {fp['dataset']}")
            print(f"Type d'échantillon: {fp['sample_type']}")
            print(f"FD: {fp['fd']}")
            print(f"Validité sur échantillon: {fp['approx_sample']:.1f}%")
            print(f"Validité sur complet: {fp['approx_full']:.1f}%")
            print(f"Violations sur dataset complet: {fp['violations_full']}")
    else:
        print("Aucun faux positif détecté.")
else:
    print("Aucun résultat disponible.")

## 11. Sauvegarder les résultats

In [ ]:
# Sauvegarder les résultats
if validation_results:
    results_df = pd.DataFrame(validation_results)
    results_df.to_csv('../results/task3_validation_results.csv', index=False)
    print("✅ Résultats sauvegardés: results/task3_validation_results.csv")

# Sauvegarder les réponses LLM
with open('../results/task3_llm_responses.json', 'w', encoding='utf-8') as f:
    json.dump(llm_suggested_fds, f, indent=2, ensure_ascii=False)
print("✅ Réponses LLM: results/task3_llm_responses.json")

# Sauvegarder les FDs parsées
parsed_fds_serializable = {}
for dataset, samples_dict in parsed_fds.items():
    parsed_fds_serializable[dataset] = {}
    for sample_type, fds in samples_dict.items():
        parsed_fds_serializable[dataset][sample_type] = fds

with open('../results/task3_parsed_fds.json', 'w', encoding='utf-8') as f:
    json.dump(parsed_fds_serializable, f, indent=2, ensure_ascii=False)
print("✅ FDs parsées: results/task3_parsed_fds.json")

## 12. Conclusion

### Ce qu'on a appris :

1. **L'échantillonnage cache des violations**
   - Une FD peut sembler vraie sur un petit échantillon
   - Mais être fausse sur le dataset complet

2. **Les LLMs généralisent trop vite**
   - Ils trouvent des patterns dans peu de données
   - Ces patterns ne sont pas forcément des règles générales

3. **Types de problèmes détectés :**
   - **Faux positifs** : FDs vraies sur échantillon, fausses sur complet
   - **FDs non minimales** : Trop d'attributs dans le LHS
   - **FDs trompeuses** : Semblent logiques mais sont accidentelles

### Leçon clé :
> Les patterns empiriques sur des échantillons ne sont PAS des contraintes.

In [ ]:
print("\n" + "="*80)
print("RÉSUMÉ FINAL - TÂCHE 3")
print("="*80)

if validation_results:
    results_df = pd.DataFrame(validation_results)
    print(f"\n✅ Datasets analysés: {results_df['dataset'].nunique()}")
    print(f"✅ FDs suggérées par LLM: {len(results_df)}")
    print(f"✅ Faux positifs détectés: {(results_df['category'] == 'false_positive').sum()}")

print(f"\n📁 Fichiers générés:")
print(f"   - results/task3_validation_results.csv")
print(f"   - results/task3_llm_responses.json")
print(f"   - results/task3_parsed_fds.json")